In [20]:
import numpy as np
import pandas as pd
import joblib
import pickle

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, log_loss

from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing import image

from PIL import Image
from scipy.spatial.distance import euclidean
import os


In [21]:
# STEP 1: Load Data
df = pd.read_csv("image_features.csv")
print(df.head())

# Extract labels and features
y = df['image'].apply(lambda name: name.split('_')[0])
X = df.drop(columns=['image'])

print(f"\n Total images in dataset: {len(X)}")


                  image         0         1         2         3         4  \
0  michael_neutral.jpeg  0.000000  2.457851  0.072053  0.685160  0.066686   
1  michael_smiling.jpeg  0.000000  0.081905  0.000000  0.364469  0.260483   
2   gustav_smiling.jpeg  0.113994  1.245440  0.000000  0.438025  0.008951   
3   gustav_neutral.jpeg  0.000000  2.675371  0.000000  0.107593  0.038977   
4  gustav_suprised.jpeg  0.237153  0.996216  0.000000  0.147609  0.039693   

          5         6         7         8  ...      1270      1271      1272  \
0  0.003916  0.160129  1.126730  0.142784  ...  1.314852  1.899158  2.787209   
1  1.053621  0.079547  0.798457  0.143555  ...  0.222954  0.045315  0.917881   
2  0.440612  0.166604  1.675608  0.000000  ...  0.114223  0.288914  3.224108   
3  0.152160  0.055685  0.849564  0.000000  ...  0.363211  0.628582  2.986023   
4  0.852565  0.492953  1.410161  0.015161  ...  0.214247  0.686016  2.989209   

       1273      1274      1275      1276      1277     

In [22]:
# STEP 2: Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [23]:
# STEP 3: Train Logistic Regression
model = LogisticRegression(max_iter=1000, multi_class='multinomial', solver='lbfgs')
model.fit(X_train, y_train)


/usr/local/lib/python3.12/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=1000, multi_class='multinomial')

In [24]:
# STEP 4: Predictions & Metrics
y_pred = model.predict(X_test)
probs = model.predict_proba(X_test)

# Accuracy & F1
acc = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='macro')
loss = log_loss(y_test, probs, labels=model.classes_)

# STEP 5: Print Everything
print("\n Classification Report:")
print(classification_report(y_test, y_pred))

print(f"\n Accuracy: {acc:.4f}")
print(f" F1 Score: {f1:.4f}")
print(f" Log Loss: {loss:.4f}")



 Classification Report:
              precision    recall  f1-score   support

     michael       0.75      1.00      0.86         3
       reine       1.00      0.75      0.86         4

    accuracy                           0.86         7
   macro avg       0.88      0.88      0.86         7
weighted avg       0.89      0.86      0.86         7


 Accuracy: 0.8571
 F1 Score: 0.8571
 Log Loss: 0.3274


In [25]:
# Save logistic regression model
joblib.dump(model, "facial_recognition_model.pkl")

# Save training features & labels for distance check
with open("known_features.pkl", "wb") as f:
    pickle.dump((X_train.to_numpy(), y_train.to_numpy()), f)


In [26]:
# Load MobileNet model
mobilenet_model = MobileNetV2(weights='imagenet', include_top=False, pooling='avg', input_shape=(224, 224, 3))

def load_and_preprocess_image(img_path):
    try:
        img = Image.open(img_path).resize((224, 224))
        img_array = np.array(img)

        # Convert grayscale to RGB
        if img_array.ndim == 2:
            img_array = np.stack((img_array,) * 3, axis=-1)
        elif img_array.shape[2] == 1:
            img_array = np.repeat(img_array, 3, axis=-1)

        img_array = preprocess_input(img_array.astype(np.float32))
        img_array = np.expand_dims(img_array, axis=0)

        return img_array
    except Exception as e:
        print(" Error loading image:", e)
        return None


In [27]:
def test_on_unseen_face(img_path, distance_threshold=0.6):
    print(f"\n🧪 Testing on image: {os.path.basename(img_path)}")

    img_tensor = load_and_preprocess_image(img_path)
    if img_tensor is None:
        return

    # Load models and known data
    model = joblib.load("facial_recognition_model.pkl")
    with open("known_features.pkl", "rb") as f:
        known_features, known_labels = pickle.load(f)

    # Step 1: Extract features
    feature_vector = mobilenet_model.predict(img_tensor)[0]

    # Step 2: Predict class using classifier
    probs = model.predict_proba([feature_vector])[0]
    predicted_index = np.argmax(probs)
    predicted_class = model.classes_[predicted_index]
    confidence = probs[predicted_index]

    print("\n Prediction Probabilities:")
    for cls, prob in zip(model.classes_, probs):
        print(f"{cls}: {prob:.2f}")

    print(f"\n Predicted: {predicted_class}")
    print(f" Confidence: {confidence:.2f}")

    # Step 3: Distance check
    distances = [euclidean(feature_vector, known_vec) for known_vec in known_features]
    min_distance = min(distances)
    closest_label = known_labels[np.argmin(distances)]

    print(f" Min Distance to known face: {min_distance:.4f}")
    print(f" Closest to: {closest_label}")

    if min_distance < distance_threshold:
        print(f" Access Granted to: {predicted_class}")
    else:
        print(" Access Denied: Unknown user")


In [28]:
test_on_unseen_face("uknown_face.jpeg")



🧪 Testing on image: uknown_face.jpeg
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step

 Prediction Probabilities:
gustav: 0.03
michael: 0.38
reine: 0.59

 Predicted: reine
 Confidence: 0.59
 Min Distance to known face: 23.5094
 Closest to: michael
 Access Denied: Unknown user


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning: X does not have valid feature names, but LogisticRegression was fitted with feature names
  warnings.warn(
